In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
 
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
 
 
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 160983 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.24-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python3.7 -m pip install nvidia-ml-py3
!python3.7 -m pip install fastai==1.0.61
!python3.7 -m pip install lpips==0.1.3
!sudo apt-get install libsvm-dev
!python3.7 -m pip install pybrisque
!python3.7 -m pip install sk-video
!python3.7 -m pip install geffnet

In [ ]:
%cd /content/
!wget https://github.com/cjlin1/libsvm/archive/refs/tags/v324.tar.gz
!tar zxvf v324.tar.gz
%cd libsvm-324/
!make clean
!make
!rm /content/v324.tar.gz

In [10]:
%cd /content/libsvm-324/
!make clean
!make

/content/libsvm-324
rm -f *~ svm.o svm-train svm-predict svm-scale libsvm.so.2
g++ -Wall -Wconversion -O3 -fPIC -c svm.cpp
g++ -Wall -Wconversion -O3 -fPIC svm-train.c svm.o -o svm-train -lm
g++ -Wall -Wconversion -O3 -fPIC svm-predict.c svm.o -o svm-predict -lm
g++ -Wall -Wconversion -O3 -fPIC svm-scale.c -o svm-scale


In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# source code in github main
%cd /content/
!cp -R /content/drive/MyDrive/ARGAN-Temp/Artifact_Removal_GAN/ /content/
if not 'Artifact_Removal_GAN' in sys.path:
  sys.path += ['Artifact_Removal_GAN']
%cd Artifact_Removal_GAN/
!mkdir models
!wget -P /content/Artifact_Removal_GAN/models/ https://github.com/mameli/Artifact_Removal_GAN/releases/download/1.1/standard.pkl

In [50]:
import sys
sys.path.append('/content/libsvm-324/python')
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

from fastai import *
from fastai.vision import *
from fastai.vision.gan import *
from ArNet.generators import *
from ArNet.critics import *
from ArNet.dataset import *
from ArNet.loss import *
from ArNet.save import *
from ArNet.fid_loss import *
from ArNet.ssim import *
from ArNet.metrics import *

import torchvision
import geffnet # efficient/ mobile net

In [ ]:
!nvidia-smi

In [74]:
def do_fit(learn, epochs,save_name, lrs=slice(1e-3), pct_start=0.9):
    learn.fit_one_cycle(epochs, lrs, pct_start=pct_start)
    learn.save(save_name)
    learn.show_results(rows=1, imgsize=20)

In [ ]:
%cd dataset/
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
!unzip DIV2K_train_HR.zip
!rm DIV2K_train_HR.zip
%cd /content/Artifact_Removal_GAN/

In [ ]:
%cd dataset/
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip
!unzip DIV2K_valid_HR.zip
!mv DIV2K_valid_HR/ DIV2K_train_HR/
!rm DIV2K_valid_HR.zip
%cd /content/Artifact_Removal_GAN/

In [ ]:
!cp /content/drive/MyDrive/MyTrainData/* dataset/DIV2K_train_HR/

In [12]:
!cp dataset/DIV2K_train_HR/*  /content/drive/MyDrive/MyOriData/

In [ ]:
!rm -R ./dataset/DIV2K_train

In [ ]:
!rm -R ./dataset/DIV2K_train_LR_256_QF20
!rm -R ./dataset/DIV2K_train_LR_512_QF20
!rm -R ./dataset/DIV2K_train_LR_Full_QF20

In [52]:
path = Path('./dataset/')

path_fullRes = path/'DIV2K_train_HR'

path_lowRes_256 = path/'DIV2K_train_LR_256_QF20'
path_lowRes_512 = path/'DIV2K_train_LR_512_QF20'
path_lowRes_Full = path/'DIV2K_train_LR_Full_QF20'

proj_id = 'unet_superRes_mobilenetV3_LPIPS'

gen_name = proj_id + '_gen'
crit_name = proj_id + '_crit'

nf_factor = 2
pct_start = 1e-8

In [16]:
datasets = [(path_lowRes_256, 256, 20)]
create_dataset(path_fullRes, datasets, downsize=False)

Creating dataset/DIV2K_train_LR_256_QF20
Size: 256 with 20 quality factor


In [ ]:
datasets = [(path_lowRes_Full, 0, 20)]
create_dataset(path_fullRes, datasets, downsize=False)

Creating dataset/DIV2K_train_LR_Full_QF20
Size: 0 with 20 quality factor


In [53]:
model = geffnet.mobilenetv3_rw

In [54]:
# loss_func = FeatureLoss()
# loss_func = msssim
# loss_func = fid
# loss_func = F.mse_loss
loss_func = lpips_loss()

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [on]
Loading model from: /usr/local/lib/python3.7/dist-packages/lpips/weights/v0.1/vgg.pth


# 256px

In [55]:
bs=10
sz=256
lr = 1e-2
wd = 1e-3
epochs = 10

In [ ]:
!pip install "torch==1.4" "torchvision==0.5.0"

In [ ]:
!pip install wandb

In [ ]:
!pip install --upgrade wandb==0.10.8

In [56]:
%cd /content/Artifact_Removal_GAN/ 

/content/Artifact_Removal_GAN


In [43]:
data_gen = get_DIV2k_data_QF(path_lowRes_256, path_fullRes, bs=bs, sz=sz)

In [57]:
data_gen = get_DIV2k_data_QF(path_lowRes_Full, path_fullRes, bs=bs, sz=sz)

In [ ]:
loss_func=loss_func.cuda()

In [69]:
learn_gen = gen_learner_wide(data=data_gen,
                             gen_loss=loss_func,
                             arch = model,
                             nf_factor=nf_factor)

In [70]:
learn_gen.metrics.append(LPIPS_Metric_gen())
learn_gen.metrics.append(LPIPS_Metric_input())

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [on]
Loading model from: /usr/local/lib/python3.7/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [on]
Loading model from: /usr/local/lib/python3.7/dist-packages/lpips/weights/v0.1/alex.pth


In [ ]:
learn_gen.metrics.append(SSIM_Metric_gen())
learn_gen.metrics.append(SSIM_Metric_input())
# learn_gen.metrics.append(LPIPS_Metric_gen())
# learn_gen.metrics.append(LPIPS_Metric_input())
learn_gen.metrics.append(BRISQUE_Metric_gen())
learn_gen.metrics.append(BRISQUE_Metric_input())
learn_gen.metrics.append(BRISQUE_Metric_target())
learn_gen.metrics.append(NIQE_Metric_gen())
learn_gen.metrics.append(NIQE_Metric_input())
learn_gen.metrics.append(NIQE_Metric_target())

In [ ]:
!print(loss_func.model.parameters())

In [71]:
wandbCallbacks = True
import datetime

if wandbCallbacks:
    import wandb
    from wandb.fastai import WandbCallback
    config={"batch_size": bs,
            "img_size": (sz, sz),
            "learning_rate": lr,
            "weight_decay": wd,
            "num_epochs": epochs
    }
    wandb.init(project='SuperRes', config=config, id="unet_superRes_mobilenetV3_FID"+ datetime.datetime.now().strftime('_%m-%d_%H:%M'))

    learn_gen.callback_fns.append(partial(WandbCallback, input_type='images'))

Problem at: <ipython-input-71-66c98a880d97> 13 <module>


UsageError: ignored

In [ ]:
!lsb_release -a

In [ ]:
learn_gen.lr_find()
learn_gen.recorder.plot()
learn_gen.summary()

epoch,train_loss,valid_loss,lpips_gen,lpips_in,time


In [ ]:
do_fit(learn_gen, 10, gen_name+"_256px_0", slice(lr*10))

In [ ]:
learn_gen.unfreeze()

In [ ]:
do_fit(learn_gen, 3, gen_name+"_256px_1", lr)

# 512px

In [ ]:
bs=2
sz=512
epochs = 1

In [ ]:
data_gen = get_DIV2k_data_QF(path_lowRes_512, path_fullRes, bs=bs, sz=sz)

In [ ]:
learn_gen.data = data_gen
learn_gen.freeze()
gc.collect()

6563

In [ ]:
# learn_gen.load(gen_name+"_256px_1")

In [ ]:
# learn_gen.lr_find()
# learn_gen.recorder.plot()

In [ ]:
print("Upsize to gen_512")

do_fit(learn_gen, 3, gen_name+"_512px_0", slice(1e-3))

In [ ]:
learn_gen.unfreeze()

In [ ]:
do_fit(learn_gen, 3, gen_name+"_512px_1", 1e-3)

In [ ]:
learn_gen.show_results(rows=10, imgsize=20)

# Quality 35

In [ ]:
bs=2
sz=512
epochs = 1

path_lowRes_512 = path/'DIV2K_train_LR_512_QF35'

In [ ]:
data_gen = get_DIV2k_data_QF(path_lowRes_512, path_fullRes, bs=bs, sz=sz)

In [ ]:
learn_gen.data = data_gen
learn_gen.freeze()
gc.collect()

26433

In [ ]:
do_fit(learn_gen, 3, gen_name+"_512px_2", 1e-3)

In [ ]:
# learn_gen.load(gen_name+"_512px_2")

In [ ]:
learn_gen.unfreeze()

In [ ]:
do_fit(learn_gen, 3, gen_name+"_512px_3", slice(1e-3))

# Quality 50

In [ ]:
bs=2
sz=512
epochs = 1

path_lowRes_512 = path/'DIV2K_train_LR_512_QF50'

In [ ]:
data_gen = get_DIV2k_data_QF(path_lowRes_512, path_fullRes, bs=bs, sz=sz)

In [ ]:
# learn_gen.data = data_gen
# learn_gen.freeze()
gc.collect()

0

In [ ]:
# learn_gen.lr_find()
# learn_gen.recorder.plot()

In [ ]:
do_fit(learn_gen, 1, gen_name+"_512px_4", 1e-3)

In [ ]:
learn_gen.load(gen_name+"_512px_4")

In [ ]:
learn_gen.unfreeze()

In [ ]:
do_fit(learn_gen, 3, gen_name+"_512px_5", slice(1e-3))

# Fine Tune patch model

In [ ]:
bs=2
sz=512
lr = 1e-2
wd = 1e-3
epochs = 1

In [ ]:
data_gen = get_DIV2k_data_QF(path_lowRes_512, path_fullRes, bs=bs, sz=sz)

In [ ]:
learn_gen = gen_learner_wide(data=data_gen,
                             gen_loss=loss_func,
                             arch = model,
                             nf_factor=nf_factor)

In [ ]:
learn_gen.metrics.append(SSIM_Metric_gen())
learn_gen.metrics.append(SSIM_Metric_input())
learn_gen.metrics.append(BRISQUE_Metric_gen())
learn_gen.metrics.append(BRISQUE_Metric_input())
learn_gen.metrics.append(BRISQUE_Metric_target())
learn_gen.metrics.append(NIQE_Metric_gen())
learn_gen.metrics.append(NIQE_Metric_input())
learn_gen.metrics.append(NIQE_Metric_target())

In [ ]:
learn_gen.load("/data/students_home/fmameli/repos/Artifact_Removal_GAN/dataset/DIV2K_train_LR_Patches/64px_FullQF20/models/unet_superRes_mobilenetV3_Patches64px_gen_64px_2")

In [ ]:
do_fit(learn_gen, 1, gen_name+"_512px_5", 1e-3)

In [ ]:
learn_gen.unfreeze()

In [ ]:
do_fit(learn_gen, 3, gen_name+"_512px_5", 1e-3)

In [ ]:
learn_gen.show_results(rows=5, imgsize=15)

# Test

In [ ]:
bs=1
sz=512
epochs = 1

In [ ]:
path_lowRes_512 = path/'DIV2K_train_LR_512_QF20'
size=( 512, 680)

In [ ]:
data_1k = (ImageImageList.from_folder(path_lowRes_512, presort=True).split_by_idxs(
            train_idx=list(range(0, 800)), valid_idx=list(range(800, 900)))
          .label_from_func(lambda x: path_fullRes/x.name.replace(".jpg", ".png"))
          .transform(get_transforms(), size=size, tfm_y=True)
          .databunch(bs=1).normalize(imagenet_stats, do_y=True))
data_1k.c = 3

In [ ]:
learn_gen.data = data_1k
learn_gen.freeze()
gc.collect()

20088

In [ ]:
learn_gen.load("/data/students_home/fmameli/repos/Artifact_Removal_GAN/dataset/DIV2K_train_LR_Patches/64px_FullQF20/models/unet_superRes_mobilenetV3_Patches64px_gen_64px_2")

In [ ]:
fn = data_1k.valid_dl.x.items[2]; fn

PosixPath('dataset/DIV2K_train_LR_512_QF20/0803.jpg')

In [ ]:
img = open_image(fn); print(img.shape)
p,img_hr,b = learn_gen.predict(img)

torch.Size([3, 512, 680])


In [ ]:
show_image(img, figsize=(15,15), interpolation='nearest');

In [ ]:
Image(img_hr).show(figsize=(15,15))